In [ ]:
# importar bibliotecas,
# tensorflow é uma biblioteca de código aberto para aprendizado de máquina 
# e aprendizado profundopara treinamento e inferência de redes neurais artificiais.
# https://www.tensorflow.org/
# %pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore


# constante do endereco do arquivo
ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

In [ ]:
try:
    print('Obtendo dados...')
    df = pd.read_csv(ENDERECO_DADOS,sep=',',encoding='utf-8')[['review_title','overall_rating']]  
# delimitando colunas para criação do modelo\n",
    df = df.dropna(subset=['review_title','overall_rating'])
    
# transformando colunas em arrays\n",
    texts = np.array(df['review_title'])
    ratings = np.array(df['overall_rating'])  
    
    print(df.head())
    print(len(df))
    print(df['overall_rating'].unique())
    print(df['overall_rating'].value_counts())
    
except Exception as e:
    print('Errro ao obter dados: ',e)

Vetorização de texto

In [ ]:
try:
    print('Vetorizando texto...')
 # Passo 1: Tokenizar os textos
    tokenizer = Tokenizer()
# fit_on_texts: cria o vocabulário
#vocabulário:dicionário que mapeia palavras para números
# cada palavra é mapeada para um número inteiro e 
# esse número inteiro é o índice da palavra no vocabulário
# toda vez que uma palavra aparece em um texto,
# ela é substituída pelo seu índice no vocabulário
    tokenizer.fit_on_texts(texts)
# Converter os textos em sequências de números
# texts_to_sequences: converte os textos em sequências de números
# É a vetorização dos textos. Cada texto é convertido em um,
# vetor de números inteiros, onde cada número inteiro representa
 # uma palavra do texto, baseada no índice da palavra no vocabulário
    vetores = tokenizer.texts_to_sequences(texts)
 
# Passo 2: Padronizar o comprimento das sequências (deixar todas com o mesmo tamanho)\n",
    padded_vetores = pad_sequences(vetores)  # Deixar todas as sequências com tamanho 10\n",
 
# Verificar o resultado da tokenização e padding\n",
    print(padded_vetores)
    
    print('Texto vetorizado com sucesso!')
   
except Exception as e:
     print('Erro ao vetorizar texto: ',e)

Construindo a rede neural...

In [ ]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Embedding, LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

# Definir as constantes para o modelo\n",
try:
    print('Construindo a rede neural...')
# tamanho do vocabulário\n",
    VOCAB_SIZE = len(tokenizer.word_index) + 1  

# Comprimento máximo da sequência\n",
    MAX_SEQUENCE_LENGTH = padded_vetores.shape[1]
   
# Tamanho do vetor\n",
    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))
    
# Construir o modelo sequencial\n",
    model = Sequential()
  
# Camada de entrada
    model.add(Embedding, input_dim=VOCAB_SIZE, input_dim=VOCAB_SIZE,output_dim=VETOR_LENGHT,input_length=MAX_SEQUENCE_LENGTH)
          
# Camada LSTM para capturar a sequência do texto
# rodar primeiro somente com 128
# model.add(LSTM(128))
    
# rodar com mais camadas
# return_sequences: parâmetro que indica se a camada deve retornar
# a sequência inteira para a próxima camada
# sequencia inteira é a sequência de saídas de cada unidade de tempo
    model.add(LSTM(128, return_sequences=True))
    
    model.add(LSTM(64, return_sequences=True))
    
# última camada LSTM não precisa retornar a sequência inteira
    model.add(LSTM(32))          
                                 
# Camada final para prever a nota (classificação)
# Definir a quantidade de neurônios de acordo 
# com a quantidade de classes/categorias
    model.add(Dense(5))
# construir o modelo\n",
    model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))
   
# Usar um otimizador Adam com uma taxa de aprendizado menor
# Adam: é um otimizador que ajusta a taxa de aprendizado
    otimizador = Adam(learning_rate=0.0001) 
  
# Compilar o modelo
# loss precisa ser ajustado de acordo com o tipo de problema
# nesse caso, é um problema de classificação
# sparse_categorical_crossentropy: função de perda para classificação
# de múltiplas classes/categorias
# sparse: indica que os rótulos são inteiros
# categorical: indica que os rótulos são one-hot encoded, ou seja,
# cada rótulo é uma categoria
# crossentropy: é a função de perda que mede a diferença entre a distribuição
# de probabilidade prevista pelo modelo e a distribuição de probabilidade real
# nesse caso observa-se q a perda durante as épocas
# Se a perda estiver diminuindo, o modelo está aprendendo
# Se a perda estiver aumentando, o modelo não está aprendendo e pode estar
# ocorrendo overfitting
    
    model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')    

    # Verificar o resumo do modelo
                   #model.summary()
except Exception as e:
         print('Erro ao construir a rede neural: ',e)                       

In [ ]:
# treinar a rede neural
try:
    print('Treinando a rede neural...')
   
# Dividir os dados em treino e teste (80% treino, 20% teste)
    from sklearn.model_selection import train_test_split # type: ignore
    X_train, X_test, y_train, y_test = train_test_split
    padded_vetores, # textos vetorizados
    ratings, # notas\n",
    test_size=0.2, # 20% dos dados para teste
    random_state=42 # seed para reprodução dos resultados
   
# Ajuste temporário dos rótulos para o intervalo [0, 4] para o treinamento
# Visto que a classificação começa em 1 e termina em 5
# e a classificação começa em 0\n",
    y_train_adjusted = y_train - 1
    y_test_adjusted = y_test - 1
   
# Aumentar o número de épocas para dar mais tempo ao modelo para aprender
# Aumentar o tamanho do batch para acelerar o treinamento, porém,
# pode ser que o modelo não aprenda bem
# Reduzir o tamanho do batch para dar mais tempo ao modelo para aprender
# e para evitar overfitting e aprender melhor
    model.fit(
            X_train, # textos de treino 
                           y_train_adjusted, # notas de treino
                           epochs=10, # número de épocas 
                            batch_size=32, # tamanho do batch de treino
                            validation_data=(X_test, y_test_adjusted)
                           )
    
except Exception as e:
        print('Erro ao treinar a rede neural: ',e)

Realizar previsões

In [ ]:
try:
    print('Realizando previsões...')
# Exemplos de novos textos para testar o modelo\n",
    novos_textos = [
                       "Muito bom, gostei bastante. Top demais! Compensa muito\",\n",
                        "Não recomendo, péssimo produto. Não funciona\",\n",
                    ]
   
# Vetorizar e padronizar os novos textos\n",
    novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
    novas_sequencias_padded = pad_sequences(novas_sequencias)
   
# Fazer previsões\n",
    predicoes = model.predict(novas_sequencias_padded)
   
# Exibir as previsões\n",
    print("Previsões:", predicoes + 1)

except Exception as e:
    print('Erro ao realizar previsões: ',e)